<a href="https://colab.research.google.com/github/tomonari-masada/course2022-sml/blob/main/09_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 糖尿病をロジスティック回帰で予測してみる

* 有名なPima Indians Diabetes Databaseを使う

 * https://www.kaggle.com/uciml/pima-indians-diabetes-database

* ロジスティック回帰、そして、分類の評価については、下記も参照
 * https://developers.google.com/machine-learning/crash-course/logistic-regression/
 * https://developers.google.com/machine-learning/crash-course/classification/

## 1) データの読み込み

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, auc, roc_curve
from sklearn.model_selection import StratifiedKFold

%config InlineBackend.figure_format = 'retina'

In [ ]:
diabetes = pd.read_csv('/content/drive/MyDrive/2021Courses/SML/diabetes.csv')

In [ ]:
diabetes.head()

In [ ]:
diabetes.info()

In [ ]:
y = diabetes['Outcome']
X = diabetes.drop('Outcome', axis=1)

In [ ]:
X.shape

In [ ]:
X.describe()

## 2) 訓練データ、テストデータに分割

* この分割は変えないようにしてください。

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=123)

* 以下、訓練データ部分を使って、交差検証によって良いモデルを探す。

---



## 3) デフォルト設定のロジスティック回帰をベースラインとみなしてテストデータでの評価値を得る


In [ ]:
logreg_baseline = LogisticRegression(random_state=123)
logreg_baseline.fit(X_train, y_train)

* `max_iter`が小さいとの警告が出ているので、増やして学習しなおし。

In [ ]:
logreg_baseline = LogisticRegression(max_iter=1000, random_state=123)
logreg_baseline.fit(X_train, y_train)

* 大丈夫だったので、テストデータでの最終評価値を得る。

In [ ]:
print(f'test score: {logreg_baseline.score(X_test, y_test):.4f}')

* Area under ROC curveも計算してみる。


In [ ]:
y_test_pred_proba = logreg_baseline.predict_proba(X_test)
print(f'ROC AUC: {roc_auc_score(y_test, y_test_pred_proba[:,1]):.4f}')

* ROC curveを描いてみる。
 * https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html#sphx-glr-auto-examples-model-selection-plot-roc-py

In [ ]:
y_score = logreg_baseline.decision_function(X_test)

fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, color='darkorange', label=f'ROC curve (area = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")

* これがベースラインなので、これより良い結果を得るべく、試行錯誤してください。
* 試行錯誤した結果として辿り着いたモデルで、最後に一回、テストデータ上での評価を行ってください。

* ロジスティック回帰についてscoreがどのように計算されているかの確認
 * thresholdが0.5である必要は、実は、ない。
 * thresholdを、交差検証で決定してもよい。

* `threshold = 0.5`とすれば、次のセルで求まる値と、上で求めたtest scoreは、一致する。

In [ ]:
threshold = 0.5
((logreg_baseline.predict_proba(X_test) >= threshold)[:,1] * 1 == y_test).sum() / len(y_test)

## 4) 交差検証しつつ試行錯誤する

* 元のデータを保存しておく。
 * 後で、訓練データだけから得られる情報を使って、訓練データも、テストデータも、変更するため。

In [ ]:
X_train_original = X_train.copy()
X_test_original = X_test.copy()

### A) 最初に交差検証の準備をしておく

In [ ]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=123)

### B) 交差検証による評価のための関数を定義しておく

In [ ]:
def cv(skf, X_train, y_train, args={}):
  scores = list()
  for train_index, valid_index in skf.split(X_train, y_train):

    # ロジスティック回帰の学習
    if not 'max_iter' in args:
      logreg = LogisticRegression(**args, max_iter=1000, random_state=123)
    else:
      logreg = LogisticRegression(**args, random_state=123)
    logreg.fit(X_train.iloc[train_index], y_train.iloc[train_index])

    # 検証データでの評価
    score = logreg.score(X_train.iloc[valid_index], y_train.iloc[valid_index])
    print(f'score: {score:.4f}')
    scores.append(score)

  print(f'mean score: {np.array(scores).mean():.4f}', end=' ')
  if args:
    print('{', ' '.join([f'"{str(k):s}":{str(args[k]):s}' for k in args]), '}')
  else:
    print()

### C) デフォルト設定のロジスティック回帰の性能を評価する
* 交差検証で性能評価するとどうなるかを確認している。

In [ ]:
cv(skf, X_train, y_train)

### D) BloodPressureへの対応

* まず、属性「BloodPressure」について、ヒストグラムを描いてよくよく眺める。


In [ ]:
sns.histplot(X_train['BloodPressure']);

* 0という値がけっこうあるらしい。実は、これは欠測値。そこで、中央値で埋めることにする。
 * この前処理の仕方で正しいという保証はありません。


In [ ]:
bp_median = np.median(X_train[X_train.BloodPressure != 0]['BloodPressure'])
print(f'blood pressure median: {bp_median}')
X_train = X_train.replace({'BloodPressure':0}, bp_median)

* もう一度、ヒストグラムを描く。


In [ ]:
sns.histplot(X_train['BloodPressure']);

* 交差検証で評価する。

In [ ]:
cv(skf, X_train, y_train)

* test dataの「BloodPressure」の欠測値も、training dataと同じ値で埋める。
 * この作業は、training dataから得られる情報しか使っていないので、ズルはしていない。


In [ ]:
X_test = X_test.replace({'BloodPressure':0}, bp_median)

### E) BMIへの対応

* 次に、training dataの「BMI」のヒストグラムを描いてみる


In [ ]:
sns.histplot(X_train['BMI']);

* やはり欠測値の部分が0とされているようなので、先ほどと同様、中央値で埋める。


In [ ]:
bmi_median = np.median(X_train[X_train.BMI != 0]['BMI'])
print(f'BMI median: {bmi_median}')
X_train = X_train.replace({'BMI':0}, bmi_median)
sns.histplot(X_train['BMI']);

In [ ]:
cv(skf, X_train, y_train)

* test dataのBMIの欠測値も、同じ値で埋める 



In [ ]:
X_test = X_test.replace({'BMI':0}, bmi_median)

### F) Glucoseへの対応

In [ ]:
sns.histplot(X_train['Glucose']);

In [ ]:
glucose_median = np.median(X_train[X_train.Glucose != 0]['Glucose'])
print(f'glucose median: {glucose_median}')
X_train = X_train.replace({'Glucose':0}, glucose_median)
sns.histplot(X_train['Glucose']);

In [ ]:
cv(skf, X_train, y_train)

* test dataのGlucoseの欠測箇所も、同じ値で埋める 



In [ ]:
X_test = X_test.replace({'Glucose':0}, glucose_median)

ここまでの交差検証でのベスト・スコアは0.7759。

### G) DiabetesPedigreeFunctionへの対応

In [ ]:
sns.histplot(X_train['DiabetesPedigreeFunction'], bins=50);

In [ ]:
from scipy.stats import boxcox

X_train_boxcox = X_train.copy()
X_train_boxcox.DiabetesPedigreeFunction, maxlog = boxcox(X_train.DiabetesPedigreeFunction)
sns.histplot(X_train_boxcox['DiabetesPedigreeFunction'], bins=50);

In [ ]:
cv(skf, X_train_boxcox, y_train)

悪くなったので不採用。

### H) SkinThicknessとInsulin: 線形回帰で欠測値を埋める

In [ ]:
sns.histplot(X_train['SkinThickness'], bins=50);

In [ ]:
sns.histplot(X_train['Insulin'], bins=50);

In [ ]:
(X_train['SkinThickness'] == 0).sum()

In [ ]:
(X_train['Insulin'] == 0).sum()

In [ ]:
((X_train['SkinThickness'] == 0) & (X_train['Insulin'] == 0)).sum()

In [ ]:
for i in X_train.index[X_train['SkinThickness'] == 0]:
  if not i in X_train.index[X_train['Insulin'] == 0]:
    print('No')

SkinThicknessが0の個体は、必ずInsulinも0になっているらしい。

ただし、これは訓練データだけでこうなっているだけかもしれないので、この事実に依存して何かをすることはしない。

In [ ]:
# LassoでSkinThicknessとInsulinの欠測部分を埋める

from sklearn.linear_model import Lasso

columns = X_train.columns[(X_train.columns != 'SkinThickness') & (X_train.columns != 'Insulin')]

for alpha in np.power(10.0, np.arange(12) - 5):

  X_train_copy = X_train.copy()

  indices = (X_train['SkinThickness'] != 0)
  reg = Lasso(alpha=alpha)
  reg.fit(X_train.loc[indices, columns], X_train.loc[indices, 'SkinThickness'])
  X_train_copy.loc[~ indices, 'SkinThickness'] = reg.predict(X_train.loc[~ indices, columns])

  indices = (X_train['Insulin'] != 0)
  reg = Lasso(alpha=alpha)
  reg.fit(X_train.loc[indices, columns], X_train.loc[indices, 'Insulin'])
  X_train_copy.loc[~ indices, 'Insulin'] = reg.predict(X_train.loc[~ indices, columns])

  print('-'*8, alpha, '-'*16)
  cv(skf, X_train_copy, y_train)

In [ ]:
# Ridge回帰でSkinThicknessとInsulinの欠測部分を埋める

from sklearn.linear_model import Ridge

columns = X_train.columns[(X_train.columns != 'SkinThickness') & (X_train.columns != 'Insulin')]

for alpha in np.power(10.0, np.arange(12) - 5):

  X_train_copy = X_train.copy()

  indices = (X_train['SkinThickness'] != 0)
  reg = Ridge(alpha=alpha)
  reg.fit(X_train.loc[indices, columns], X_train.loc[indices, 'SkinThickness'])
  X_train_copy.loc[~ indices, 'SkinThickness'] = reg.predict(X_train.loc[~ indices, columns])

  indices = (X_train['Insulin'] != 0)
  reg = Ridge(alpha=alpha)
  reg.fit(X_train.loc[indices, columns], X_train.loc[indices, 'Insulin'])
  X_train_copy.loc[~ indices, 'Insulin'] = reg.predict(X_train.loc[~ indices, columns])

  print('-'*8, alpha, '-'*16)
  cv(skf, X_train_copy, y_train)

悪くなったので不採用。

### I) SkinThicknessとInsulin: それぞれk-近傍法で値を埋める

In [ ]:
# k-NNでSkinThicknessの欠測部分を埋める

from sklearn.neighbors import KNeighborsClassifier

columns = X_train.columns[(X_train.columns != 'SkinThickness') & (X_train.columns != 'Insulin')]

for k in range(1, 21):

  X_train_copy = X_train.copy()

  indices = (X_train['SkinThickness'] != 0)
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(X_train.loc[indices, columns], X_train.loc[indices, 'SkinThickness'])
  X_train_copy.loc[~ indices, 'SkinThickness'] = knn.predict(X_train.loc[~ indices, columns])

  indices = (X_train['Insulin'] != 0)
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(X_train.loc[indices, columns], X_train.loc[indices, 'Insulin'])
  X_train_copy.loc[~ indices, 'Insulin'] = knn.predict(X_train.loc[~ indices, columns])

  print('-'*8, k, '-'*16)
  cv(skf, X_train_copy, y_train)

In [ ]:
# k=10を採用

k = 10

X_train_copy = X_train.copy()

indices = (X_train['SkinThickness'] != 0)
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train.loc[indices, columns], X_train.loc[indices, 'SkinThickness'])
X_train_copy.loc[~ indices, 'SkinThickness'] = knn.predict(X_train.loc[~ indices, columns])

indices = (X_train['Insulin'] != 0)
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train.loc[indices, columns], X_train.loc[indices, 'Insulin'])
X_train_copy.loc[~ indices, 'Insulin'] = knn.predict(X_train.loc[~ indices, columns])

In [ ]:
sns.histplot(X_train_copy['SkinThickness'], bins=50);

In [ ]:
sns.histplot(X_train_copy['Insulin'], bins=50);

In [ ]:
cv(skf, X_train_copy, y_train)

In [ ]:
# k=10を使って、テストデータの欠測箇所も埋めておく

X_test_copy = X_test.copy()

k = 10

indices = (X_train['SkinThickness'] != 0)
test_missing = (X_test['SkinThickness'] == 0)
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train.loc[indices, columns], X_train.loc[indices, 'SkinThickness'])
X_test_copy.loc[test_missing, 'SkinThickness'] = knn.predict(X_test.loc[test_missing, columns])

indices = (X_train['Insulin'] != 0)
test_missing = (X_test['Insulin'] == 0)
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train.loc[indices, columns], X_train.loc[indices, 'Insulin'])
X_test_copy.loc[test_missing, 'Insulin'] = knn.predict(X_test.loc[test_missing, columns])

In [ ]:
print((X_test_copy.SkinThickness == 0).sum())
print((X_test_copy.Insulin == 0).sum())

* 分類性能が良くなったので、埋めた後のデータセットを採用する。

In [ ]:
X_train = X_train_copy
X_test = X_test_copy

In [ ]:
X_train.describe()

### J) スケーラーを使ってみる。

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_temp = X_train.copy()
X_train_temp[X_train.columns] = scaler.transform(X_train)
cv(skf, X_train_temp, y_train)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_temp = X_train.copy()
X_train_temp[X_train.columns] = scaler.transform(X_train)
cv(skf, X_train_temp, y_train)

いずれも不採用。

### K) 正則化パラメータCをチューニングする。

In [ ]:
for C in np.power(10.0, np.arange(13) - 5):
  cv(skf, X_train, y_train, {'C':C})

In [ ]:
for C in np.power(10.0, np.arange(13) - 5):
  cv(skf, X_train, y_train, {'C':C, 'penalty':'l1', 'solver':'liblinear', 'max_iter':5000})

デフォルトの設定を採用。

In [ ]:
cv(skf, X_train, y_train)

### L) 分類のthresholdをチューニングしてみる

In [ ]:
def cv_threshold(skf, X_train, y_train, threshold=0.5, args={}):
  scores = list()
  for train_index, valid_index in skf.split(X_train, y_train):

    # ロジスティック回帰の学習
    if not 'max_iter' in args:
      logreg = LogisticRegression(**args, max_iter=1000, random_state=123)
    else:
      logreg = LogisticRegression(**args, random_state=123)
    logreg.fit(X_train.iloc[train_index], y_train.iloc[train_index])

    # 検証データでの評価（thresholdの値を利用している）
    X_valid = X_train.iloc[valid_index]
    y_valid = y_train.iloc[valid_index]
    score = ((logreg.predict_proba(X_valid) >= threshold)[:,1] * 1 == y_valid).sum() / len(y_valid)
    scores.append(score)

  print(f'{threshold:.2f} ; mean score: {np.array(scores).mean():.4f}', end=' ')
  if args:
    print('{', ' '.join([f'"{str(k):s}":{str(args[k]):s}' for k in args]), '}')
  else:
    print()

In [ ]:
for threshold in np.arange(4, 7, 0.5) * 0.1:
  cv_threshold(skf, X_train_original, y_train, threshold=threshold)

In [ ]:
for threshold in np.arange(4, 7, 0.5) * 0.1:
  cv_threshold(skf, X_train, y_train, threshold=threshold)

* 以上をまとめると、モデルはデフォルトのままでよく、データの欠損に対処しただけ。

### M) テストデータで最終評価

In [ ]:
logreg = LogisticRegression(max_iter=1000, random_state=123)
logreg.fit(X_train, y_train)
print('test score: {:.4f}'.format(logreg.score(X_test, y_test)))

* 分類精度だけを見ると、欠損に対処しないほうがよかった、という結論になるが・・・

In [ ]:
y_test_pred_proba = logreg.predict_proba(X_test)
print('ROC AUC: {:.4f}'.format(roc_auc_score(y_test, y_test_pred_proba[:,1])))

* AUCは改善されている。

In [ ]:
y_score = logreg_baseline.decision_function(X_test_original) # ベースラインには元のテストデータを使う
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

y_score_ours = logreg.decision_function(X_test)
fpr_ours, tpr_ours, _ = roc_curve(y_test, y_score_ours)
roc_auc_ours = auc(fpr_ours, tpr_ours)

plt.plot(fpr, tpr, color='darkorange', label=f'ROC curve (area = {roc_auc:.4f})')
plt.plot(fpr_ours, tpr_ours, color='firebrick', label=f'ROC curve (area = {roc_auc_ours:.4f})')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right");

# 課題
* 上の結果を改良できるかどうか、試行錯誤してみてください。